In [9]:
from langchain_community.document_loaders import PyPDFLoader
import pprint
loader = PyPDFLoader('attention.pdf')


In [10]:
docs = loader.load()

In [17]:
print(len(docs))  # Print the number of documents loaded
pprint.pp(docs[10].page_content)

15
('[5] Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, '
 'Holger Schwenk,\n'
 'and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder '
 'for statistical\n'
 'machine translation. CoRR, abs/1406.1078, 2014.\n'
 '[6] Francois Chollet. Xception: Deep learning with depthwise separable '
 'convolutions. arXiv\n'
 'preprint arXiv:1610.02357, 2016.\n'
 '[7] Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. '
 'Empirical evaluation\n'
 'of gated recurrent neural networks on sequence modeling. CoRR, '
 'abs/1412.3555, 2014.\n'
 '[8] Chris Dyer, Adhiguna Kuncoro, Miguel Ballesteros, and Noah A. Smith. '
 'Recurrent neural\n'
 'network grammars. In Proc. of NAACL, 2016.\n'
 '[9] Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. '
 'Dauphin. Convolu-\n'
 'tional sequence to sequence learning. arXiv preprint arXiv:1705.03122v2, '
 '2017.\n'
 '[10] Alex Graves. Generating sequences with recurrent neural networks. a

In [12]:
pprint.pp(docs[0])

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszka

In [19]:
!pip show langchain

Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

In [25]:
print(len(docs))

5


In [26]:
!pip show langchain_community

Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [30]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [29]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    23 hours ago    
llama3.2:latest            a80c4f17acd5    2.0 GB    5 weeks ago     
llama3.2:1b                baf6a787fdff    1.3 GB    5 weeks ago     


In [31]:
db = FAISS.from_documents(docs[:20], embeddings)


In [42]:
query = "An attention function can be described as a mapping from a query"
results = db.similarity_search(query)

In [44]:
pprint.pp(results[0].page_content)

('mechanism. We propose a new simple network architecture, the Transformer,\n'
 'based solely on attention mechanisms, dispensing with recurrence and '
 'convolutions\n'
 'entirely. Experiments on two machine translation tasks show these models to\n'
 'be superior in quality while being more parallelizable and requiring '
 'significantly\n'
 'less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\n'
 'to-German translation task, improving over the existing best results, '
 'including\n'
 'ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation '
 'task,\n'
 'our model establishes a new single-model state-of-the-art BLEU score of 41.8 '
 'after\n'
 'training for 3.5 days on eight GPUs, a small fraction of the training costs '
 'of the\n'
 'best models from the literature. We show that the Transformer generalizes '
 'well to\n'
 'other tasks by applying it successfully to English constituency parsing both '
 'with\n'
 'large and limited training data.'

In [91]:
!pip show langchain-ollama

Name: langchain-ollama
Version: 1.0.1
Summary: An integration package connecting Ollama and LangChain
Home-page: https://docs.langchain.com/oss/python/integrations/providers/ollama
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages
Requires: langchain-core, ollama
Required-by: 


In [94]:
import requests
import json

OLLAMA_URL = "http://localhost:11434" # Default Ollama URL

def get_ollama_models():
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags")
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()
        models = [model['name'] for model in data.get('models', [])]
        return models
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Ollama: {e}")
        return []

# Get and print the list
available_models = get_ollama_models()
print("Ollama Models:", available_models)

Ollama Models: ['nomic-embed-text:latest', 'llama3.2:latest', 'llama3.2:1b']


In [96]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2:1b")


In [49]:
!pip show langchain_core

Name: langchain-core
Version: 1.2.4
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions, uuid-utils
Required-by: langchain, langchain-classic, langchain-community, langchain-ollama, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt, langserve


In [64]:
from langchain_core.prompts import ChatMessagePromptTemplate
prompt = ChatMessagePromptTemplate.from_template("""
                                                 you are a helpful assistant that will help answer user questions
                                                 based on the context provided.
                                                 answer the questions correctly and i will give rs 200
                                                 {context}
                                                 Question: {question}
                                                 Answer:""")


ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  Field required [type=missing, input_value={'prompt': PromptTemplate...              Answer:')}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [97]:
from langchain_core.prompts import ChatMessagePromptTemplate,ChatPromptTemplate


# prompt = ChatMessagePromptTemplate.from_template(template="""
# You are a helpful assistant that helps people find information.
# Answer the following question based on the context provided.

# <context>
# {context}
# </context>

# Question: {question}
# """,
#     role="system")



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's question based on the provided context: {context}"),
        ("human", "{input}"),
    ]
)



                                                 

In [98]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm = llm, prompt = prompt)

In [99]:
retriever=db.as_retriever(document_chain=document_chain)
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10cf1f4c0>, search_kwargs={})

In [100]:
from langchain_classic.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

query = "Explain attention mechanism in neural networks."

In [102]:
answer = retrieval_chain.invoke({"input": query})

In [106]:
pprint.pp(answer['answer'])

('The Transformer model proposed by Ashish Vaswani, Noam Shazeer, Niki Parmar, '
 'Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia '
 'Polosukhin is a new simple network architecture based solely on attention '
 'mechanisms, without the need for recurrence or convolutional neural '
 'networks.\n'
 '\n'
 'In essence, this means that instead of processing sequential data in rows '
 '(recurrence) or columns (convolutions), the Transformer model processes the '
 'input sequence as a whole, using self-attention mechanisms to weigh the '
 'importance of different parts of the input. This allows for parallelization '
 'and significant reduction in training time compared to traditional models.\n'
 '\n'
 'The experiments conducted on two machine translation tasks showed that this '
 'approach can achieve superior quality while being more parallelizable and '
 'requiring less time to train than existing state-of-the-art models from the '
 'literature.')
